# July 26th, 2022 Historic Flash Flooding in St. Louis

- https://storymaps.arcgis.com/stories/9d10335079444c159966e0a28c90c4df

# Introduction

- In this activity, we'll explore the process of extracting raster data from sources like NWS, landcover, SAR data, DEM, and its related derivatives. 
- We'll then transform these layers into a structured tabular format.
- With the data in this form, we will train a model in BigQuery to using a temporal predictor of weather to predict flood. 
- This will enable us to forecast flooded areas, and generate comprehensive reports detailing the impact on populations and infrastructure.

## Setup

### Environment

Clone Repo from https://github.com/objectcomputing/GEE-Geospatial-Workshop.git

For this Jupyter notebook to work, the user must set up the proper environment:

```
pip install -r requirements.txt
```

These steps will install all python modules required for this notebook

### Authentication

Setup Instructions were sent to participants prior to this session outlining registration to Google tools and Google Earth Engine.  

#### Google Cloud Tools Authentication

Individuals will automatically have access to Google tools, e.g. Cloud Storage and BigQuery APIs, and saved resources, e.g. data layers and models, in this workshop's project `gee-workshop-042023`.  In general, users of the Vertex AI Jupyterlab notebook are automatically authenticated with Google.  The expectation is that participants will establish and manage their own google projects going forward.  As such, to authenticate with google resources outside of the Vertex AI platform, it is recommended that users leverage Google's Cloud [SDK](https://cloud.google.com/sdk).

#### Google Earth Engine Authentication

GEE authentication is easy.  Run the following line and follow the necessary steps: 


In [4]:
# Import the google earth engine module and authenticate
import ee
ee.Authenticate()

## GEE Flood in St. Louis Area: Predictor Layers

For this portion of the workshop, we highlight accessing and visualizing content from the GEE [collection](https://developers.google.com/earth-engine/datasets) & derived datasets.

### Pulling and Visualizing GEE imagery

In [5]:
# Initializing Google Earth Engine
ee.Initialize()

### Response Variable: SAR 

Utilizing Sentinel 1 GRD data, we'll analyze temporal snapshots both pre and post-natural disaster to assess and quantify flood impacts.

    Data: Sentinel 1 GRD, VV band.
    Masks:
        Flood: Changes from higher to lower reflectance.
        Water: Consistent low reflectance.
    Elevation: Used USGS 3D data to filter high areas.
    High Terrain Filter: Excluded regions average elevation.

In [7]:
# Importing packages to map data layers
import folium
from utils import add_ee_layer
folium.Map.add_ee_layer = add_ee_layer

In [8]:
# Using Synthetic Aperture Radar, SAR, to detect presence of water--VV band is used
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD")

# St. Louis AOI
aoi = ee.Geometry.Point(-90.38629668465005,38.686462030922186).buffer(15000) 

# Before and After Sar Imagery using general filter for now--capturing a smaller area for demo purposes
sar_before = sentinel1.filterDate('2022-07-10', '2022-07-25').filterBounds(aoi).select('VV').mean().focal_mean(100, 'circle', 'meters').clip(aoi)
sar_after = sentinel1.filterDate('2022-07-26', '2022-08-20').filterBounds(aoi).select('VV').mean().focal_mean(100, 'circle', 'meters').clip(aoi)

# Note that the date of SAR imagery is well after the historic rain event that occurred July 26, 2022

# Flood and water masks
flood = sar_before.gt(-16).And(sar_after.lt(-16))
flood_mask = flood.updateMask(flood.eq(1))

water = sar_before.lt(-16).And(sar_after.lt(-16))  
water_mask = water.updateMask(water.eq(1))

# Get DEM to incorporate with SAR. Filter regions with high elevation
dem = ee.Image('USGS/3DEP/10m')
elev = dem.reduceRegion(ee.Reducer.mean(), aoi, 10).get('elevation').getInfo()

# Apply DEM filtering 
flood_mask = flood_mask.updateMask(dem.lt(elev-5))
water_mask = water_mask.updateMask(dem.lt(elev-5))

mymap = folium.Map(location=[38.67132182868667, -90.34667041449894], zoom_start=11)

mymap.add_ee_layer(sar_before, {'min': -25,'max': 5,'gamma': 1.0}, 'Sentinel-1 SAR Before')
mymap.add_ee_layer(sar_after, {'min': -25,'max': 5,'gamma': 1.0}, 'Sentinel-1 SAR After') 
mymap.add_ee_layer(flood_mask, {'palette':['Yellow']}, 'Flood Inundation')
mymap.add_ee_layer(water_mask, {'palette':['Blue']}, 'Water ')

folium.LayerControl().add_to(mymap)
display(mymap)

Many predictive datalayers can go into the creation of an ML model of flood.  We start with four main categories: elevation, hydrological derivatives of elevation, landcover, and precipitation.  Elevation was secured from Google Earth Engine's collection, imported into ArcGIS for derivative calculations, then exported to Google Storage.

### Pulling Elevation Data and its Derivatives

ArcGIS Pro was leveraged to derive hydrological features of elevation.  Here are references of how to do this.
- Slope: https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/slope.htm
- Flow Direction: https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/flow-direction.htm
- Flow Accumulation: https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/how-flow-accumulation-works.htm
- Streams (Reclassify High Values > 10% of max): https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/reclassify.htm
- Stream Distance (euclidean distance from streams): https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/euclidean-distance.htm

Layers have been saved off in Google Cloud Storage.  Normally, a Cloud Storage Client will need to be set up.  Vertex AI has already done this.

In [14]:
# Importing storage module to access elevation layers
from google.cloud import storage

In [16]:
# Creating Google Earth Engine Images of DEM and Hydrology layers
dem = ee.Image.loadGeoTIFF('gs://stlouis-workshop/flood/arcgis-data/cog/stl-dem.tif')
slope = ee.Image.loadGeoTIFF('gs://stlouis-workshop/flood/arcgis-data/cog/stl-slope.tif')
flow_accumulation = ee.Image.loadGeoTIFF('gs://stlouis-workshop/flood/arcgis-data/cog/stl-flow-acc.tif')
stream_distance = ee.Image.loadGeoTIFF('gs://stlouis-workshop/flood/arcgis-data/cog/distance-to-streams.tif')

stream_distance.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B0',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [17191, 9993],
   'crs': 'EPSG:4269',
   'crs_transform': [8.983152799999994e-05,
    0,
    -91.23346837809,
    0,
    -8.983152799999978e-05,
    39.156262310397004]}]}

### Pulling Landcover Data from Google Earth Engine

Details about this data layer can be found [here](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200).

In [17]:
# Landcover
landCover = ee.ImageCollection('ESA/WorldCover/v200').first().select('Map').clip(aoi)

### Pulling and Manipulating Precipitation from Cloud Storage

Precipitation data was obtained from NWS:
    https://water.weather.gov/precip/download.php
    
![NWS Precip Download](nws_precip_download.png)

The new QPE GeoTIFFs generated from the NCEP Stage IV data are multi-band GeoTIFF. The bands they contain are:

    Band 1 - Observation - Last 24 hours of QPE spanning 12Z to 12Z in inches
    Band 2 - PRISM normals - PRISM normals in inches
    Band 3 - Departure from normal - The departure from normal in inches
    Band 4 - Percent of normal - The percent of normal
    
The model of flood uses two days of precipitation leading up to the flood event.  We've already saved these layers for the workshop.  It is available in Google Storage.

In [19]:
# 1 Day Precip Data
precip_1day_0726 = ee.Image.loadGeoTIFF('gs://stlouis-workshop/flood/nws_precip_1day_20220726_conus_cog.tif')
precip_1day_0727 = ee.Image.loadGeoTIFF('gs://stlouis-workshop/flood/nws_precip_1day_20220727_conus_cog.tif')

# Precip Data Accumulation (sum up band values) - Sum up precip accumulation over multiple days of choice
precip_2day_0727 = precip_1day_0726.add(precip_1day_0727).select('B0') # select only precip accumulation, Band 1

# Precip Data Accumulation saved in GS Bucket
precip_1day_0727 = ee.Image.loadGeoTIFF('gs://stlouis-workshop/flood/arcgis-data/precip_2day_0727.tif')

Now that all data layers have been ingested, to investigate them, run the following script.

In [20]:
# Create plot of all layers
mymap = folium.Map(location=[38.67132182868667, -90.34667041449894], zoom_start=11)

# precipitation data
mymap.add_ee_layer(precip_2day_0727.clip(aoi), 
                   {'min': 0, 'max': 10.713582992553711, 
                    'palette': ['black','blue','green', 'yellow', 'orange', 'red', 'darkred', 'white']}, 
                   'precip_2day_0727')

# DEM data & its derivations
mymap.add_ee_layer(dem.clip(aoi), {
    'min': 0,
    'max': 310,
    'palette': [
      '3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13',
      'ff6e08', 'ff500d', 'ff0000', 'de0101', 'c21301', '0602ff', '235cb1',
      '307ef3', '269db1', '30c8e2', '32d3ef', '3be285', '3ff38f', '86e26f'
  ],
    'opacity': 1
}, 'dem')
mymap.add_ee_layer(slope.clip(aoi), {}, 'slope')
mymap.add_ee_layer(flow_accumulation.clip(aoi), {}, 'flow_accumulation')
mymap.add_ee_layer(stream_distance.clip(aoi), {}, 'stream_distance')

# Landcover
mymap.add_ee_layer(landCover.clip(aoi), {}, 'Landcover 10m')

# SAR
mymap.add_ee_layer(sar_before, {'min': -25,'max': 5,'gamma': 1.0}, 'Sentinel-1 SAR Before')
mymap.add_ee_layer(sar_after, {'min': -25,'max': 5,'gamma': 1.0}, 'Sentinel-1 SAR After') 
mymap.add_ee_layer(flood_mask, {'palette':['Yellow']}, 'Flood Inundation')
mymap.add_ee_layer(water_mask, {'palette':['Blue']}, 'Water ')

folium.LayerControl().add_to(mymap)
display(mymap)

In [21]:
# save map if desired
mymap.save('stlouis_map.html')

## Converting Satellite Data into a Table for ML Model

    Purpose: Transform detailed satellite data into a table format to be ingested to BQ
    Steps:
        Connection: Link up with Google Cloud Storage to access the satellite data.
        Read Data: Load the satellite image into the system's memory.
        Coordinate Calculations: Figure out the exact location of each pixel in the image.
        Polygon Creation: For every pixel, calculate its exact boundaries.
        Conversion: Turn the detailed satellite data, alongside the coordinates and boundaries, into a table.
    Result: A table where each row represents a piece of the satellite image, detailing its value, location, and shape.
    
Object Computing Inc. has created a module to simplify this step.  It is called `raster_to_tabular`.

In [22]:
from utils import raster_to_tabular

# Usage - takes a while depending on the size of the AOI
bucket_name = "stlouis-workshop"
file_name = "flood/flood-composite-data.tif"
band_names = ['dem',
              'dem_filled',
              'slope',
              'hillshade',
              'flow_direction',
              'flow_accumulation',
              'stream_distance',
              'landCover',
              'precip_2day_0727',
              'precip_0726',
              'precip_0727',
              'sar_before',
              'sar_after',
              'flood_mask',
              'water_mask']

df_result = raster_to_tabular(bucket_name=bucket_name, file_name=file_name, band_names=band_names)
df_result.head()

INFO:root:Starting raster to tabular conversion...
INFO:root:Connecting to Google Cloud Storage...
INFO:root:Reading raster into memory...
INFO:rasterio._filepath:Object not found in virtual filesystem: filename=b'97081ffe-50bf-4bfb-9e7b-915723b3f5c6/97081ffe-50bf-4bfb-9e7b-915723b3f5c6.aux'
INFO:rasterio._filepath:Object not found in virtual filesystem: filename=b'97081ffe-50bf-4bfb-9e7b-915723b3f5c6/97081ffe-50bf-4bfb-9e7b-915723b3f5c6.AUX'
INFO:rasterio._filepath:Object not found in virtual filesystem: filename=b'97081ffe-50bf-4bfb-9e7b-915723b3f5c6/97081ffe-50bf-4bfb-9e7b-915723b3f5c6.aux'
INFO:rasterio._filepath:Object not found in virtual filesystem: filename=b'97081ffe-50bf-4bfb-9e7b-915723b3f5c6/97081ffe-50bf-4bfb-9e7b-915723b3f5c6.AUX'
INFO:root:Calculating x and y coordinates...
INFO:root:Calculating corners of each pixel...
INFO:root:Flattening and transposing arrays...
INFO:root:Calculating centroids and creating polygons...


Creating polygons:   0%|          | 0/11497130 [00:00<?, ?it/s]

INFO:root:Converting to DataFrame...
INFO:root:Completed raster to tabular conversion.


,dem,dem_filled,slope,hillshade,flow_direction,flow_accumulation,stream_distance,landCover,precip_2day_0727,precip_0726,precip_0727,sar_before,sar_after,flood_mask,water_mask,x,y,geometry
0,129.431412,131.434937,0.348450,0.0,32.0,458.0,1.342821,40.0,8.303149,6.633858,1.669291,NaN,NaN,NaN,NaN,-90.557801,38.821415,"POLYGON ((-90.5578454529037 38.82146020400565,..."
1,129.386017,131.434937,0.327565,0.0,32.0,177.0,1.342893,40.0,8.303149,6.633858,1.669291,NaN,NaN,NaN,NaN,-90.557711,38.821415,POLYGON ((-90.55775562137529 38.82146020400565...
2,129.356323,131.434937,0.337773,0.0,16.0,4.0,1.342966,40.0,8.303149,6.633858,1.669291,NaN,NaN,NaN,NaN,-90.557621,38.821415,POLYGON ((-90.55766578984688 38.82146020400565...
3,129.314468,131.434937,0.278300,0.0,16.0,3.0,1.343040,40.0,8.303149,6.633858,1.669291,NaN,NaN,NaN,NaN,-90.557531,38.821415,POLYGON ((-90.55757595831847 38.82146020400565...
4,129.301712,131.434937,0.235698,0.0,16.0,2.0,1.343114,40.0,8.303149,6.633858,1.669291,NaN,NaN,NaN,NaN,-90.557441,38.821415,POLYGON ((-90.55748612679005 38.82146020400565...


## Loading a Pandas Dataframe into BigQuery

In [ ]:
from google.cloud import bigquery
from pandas_gbq import to_gbq

# Define the BigQuery parameters
project_id = 'gee-workshop-042023'
table_id = 'flood_model_demo.composite_table_sample'

# Define your schema
table_schema = [
    {"name": "dem", "type": "FLOAT"},
    {"name": "slope", "type": "FLOAT"},
    {"name": "flow_accumulation", "type": "FLOAT"},
    {"name": "stream_distance", "type": "FLOAT"},
    {"name": "landCover", "type": "FLOAT"},
    {"name": "precip_2day", "type": "FLOAT"},
    {"name": "sar_before", "type": "FLOAT"},
    {"name": "sar_after", "type": "FLOAT"},
    {"name": "flood_mask", "type": "FLOAT"},
    {"name": "water_mask", "type": "FLOAT"},
    {"name": "x", "type": "FLOAT"},
    {"name": "y", "type": "FLOAT"},
    {"name": "geometry", "type": "STRING"}
]

# Upload the DataFrame with the specified schema
to_gbq(df_result, table_id, project_id=project_id, if_exists='replace', table_schema=table_schema)

## Building BigQuery ML Model of Flood

The following [code](https://console.cloud.google.com/bigquery?sq=962299647445:d9a4b11188eb46f287d2bb4c14e86b70) will kick-off an ML model build.  However, it should not be used for production as the training SAR is out of window.  Object Computing Inc. has created a better model, and the artifact is accessible here: `gee-workshop-042023.flood_model_houston.flood_random_forest_dem_norm_2day_00`

In [ ]:
# To run the model training from a notebook, follow this code: 

from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client(project='gee-workshop-042023')

# Define the SQL query to train a model (specify model name, hyperparameters, and composite table)
query = """
CREATE OR REPLACE MODEL `gee-workshop-042023.flood_model_demo.flood_random_forest_2day_demo`
OPTIONS(
  MODEL_TYPE='RANDOM_FOREST_CLASSIFIER',
  NUM_PARALLEL_TREE=50,
  TREE_METHOD='HIST',
  MIN_TREE_CHILD_WEIGHT=5,
  COLSAMPLE_BYTREE=0.6,
  COLSAMPLE_BYLEVEL=0.8,
  COLSAMPLE_BYNODE=0.8,
  MIN_SPLIT_LOSS=0.05,
  MAX_TREE_DEPTH=10,
  SUBSAMPLE=0.9,
  AUTO_CLASS_WEIGHTS=TRUE,
  L1_REG=0.05,
  L2_REG=0.005,
  INPUT_LABEL_COLS=['flood_mask'],
  MIN_REL_PROGRESS=0.0005,
  DATA_SPLIT_METHOD='RANDOM',
  DATA_SPLIT_EVAL_FRACTION=0.2
) AS
SELECT * EXCEPT(sar_after,water_mask,geometry,x,y)
FROM `gee-workshop-042023.flood_model_demo.stl_composite_data`
WHERE RAND() <= 0.05;
"""

# Execute the query
client.query(query).result()

## ML Model Results in BQ

After BigQuery finishes training the model, results can be visualized in the console.  There are built-in model metrics and visualizations.

- Feature Importance: https://console.cloud.google.com/bigquery?sq=962299647445:759c098320c04c7d86ac70aba7e14256
- Prediction: https://console.cloud.google.com/bigquery?sq=962299647445:87ce66762280415c8e03685ce9e4e55d

### Feature Importance

Feature importance can be calculated with the following script.

In [ ]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client(project='gee-workshop-042023')

# Define the SQL query for feature importance
feature_importance_query = """
SELECT * 
FROM ML.FEATURE_IMPORTANCE(MODEL `gee-workshop-042023.flood_model_houston.flood_random_forest_dem_norm_2day_00`)
"""

# Execute the query
client.query(feature_importance_query).to_dataframe()

### Model Predictions

Using a better trained model, we'll generate predictions of flood for the St. Louis area.  Here is the script to do this:

In [ ]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client(project='gee-workshop-042023')

# Define the SQL query for ML prediction and store the result in a table 
predict_query = """
CREATE OR REPLACE TABLE `gee-workshop-042023.flood_model_demo.prediction_demo_sample` AS
SELECT
  *
FROM
  ML.PREDICT(MODEL `gee-workshop-042023.flood_model_houston.flood_random_forest_dem_norm_2day_00`, 
  (SELECT * FROM `gee-workshop-042023.flood_model_demo.stl_composite_data`))
"""

# Execute the query
client.query(predict_query).result()

## St. Louis Predictions of Flood

In [24]:
import branca

# flood prediction data
flood_prediction = ee.Image.loadGeoTIFF('gs://stlouis-workshop/flood/stl_p100_5m_cog.tif' )

# Exclude 0 values from flood prediction
flood_prediction_risk = flood_prediction.updateMask(flood_prediction.neq(0))

# Mask out values less than threshold for actual flood
flood_prediction = flood_prediction.updateMask(flood_prediction.gte(0.3))


# Create plot prediction
mymap = folium.Map(location=[38.67132182868667, -90.34667041449894], zoom_start=11)
mymap.add_ee_layer(flood_prediction_risk, 
                   {'min': 0.0,'max': 0.5,'palette': ['green','green', 'yellow', 'orange', 'red']}, 
                   'STL Flood Risk Map')
mymap.add_ee_layer(flood_prediction, 
                   {'min': 0.5,'max': 1.0,'palette': ['blue']}, 
                   'STL Flood Prediction')
folium.LayerControl().add_to(mymap)

# Define the custom legend
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; width: 150px; height: 180px; background-color: white; border:2px solid grey; z-index:9999; font-size:8px;">
<p style="padding:5px;">Legend:</p>
<p style="margin-left:10px; margin-right:10px;"><span style="background-color:green; padding:5px;">&nbsp;</span> Low Risk</p>
<p style="margin-left:10px; margin-right:10px;"><span style="background-color:yellowgreen; padding:5px;">&nbsp;</span> Moderate Risk</p>
<p style="margin-left:10px; margin-right:10px;"><span style="background-color:yellow; padding:5px;">&nbsp;</span> Medium Risk</p>
<p style="margin-left:10px; margin-right:10px;"><span style="background-color:orange; padding:5px;">&nbsp;</span>High Risk</p>
<p style="margin-left:10px; margin-right:10px;"><span style="background-color:red; padding:5px;">&nbsp;</span>Extreme Risk</p>
<p style="margin-left:10px; margin-right:10px;"><span style="background-color:blue; padding:5px;">&nbsp;</span>Flood Prediction</p>
</div>
'''

# Add the custom legend to the map
mymap.get_root().html.add_child(branca.element.Element(legend_html))


display(mymap)